In [3]:
from modelscope.pipelines import pipeline
from modelscope.utils.constant import Tasks

2023-09-09 14:03:49,718 - modelscope - INFO - Model revision not specified, use the latest revision: v1.0.2
2023-09-09 14:03:50,384 - modelscope - INFO - initiate model from /home/yangwenhao/.cache/modelscope/hub/damo/speech_campplus_sv_en_voxceleb_16k
2023-09-09 14:03:50,385 - modelscope - INFO - initiate model from location /home/yangwenhao/.cache/modelscope/hub/damo/speech_campplus_sv_en_voxceleb_16k.
2023-09-09 14:03:50,388 - modelscope - INFO - initialize model from /home/yangwenhao/.cache/modelscope/hub/damo/speech_campplus_sv_en_voxceleb_16k
2023-09-09 14:03:53,155 - modelscope - WARNING - No preprocessor field found in cfg.
2023-09-09 14:03:53,158 - modelscope - WARNING - No val key and type key found in preprocessor domain of configuration.json file.
2023-09-09 14:03:53,159 - modelscope - WARNING - Cannot find available config to build preprocessor at mode inference, current config: {'model_dir': '/home/yangwenhao/.cache/modelscope/hub/damo/speech_campplus_sv_en_voxceleb_16k'}

In [ ]:
# cam_pp
sv_pipeline = pipeline(
    task=Tasks.speaker_verification,
    model='damo/speech_campplus_sv_en_voxceleb_16k'
)

In [16]:
# ecapa-tdnn
sv_pipeline = pipeline(
    task='speaker-verification',
    model='damo/speech_ecapa-tdnn_sv_en_voxceleb_16k'
)

2023-09-09 14:42:19,347 - modelscope - INFO - Model revision not specified, use the latest revision: v1.0.1
Downloading: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 357/357 [00:00<00:00, 29.8kB/s]
Downloading: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 79.5M/79.5M [00:00<00:00, 86.6MB/s]
Downloading: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 79.5k/79.5k [00:00<00:00, 5.10MB/s]
Downloading: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

cuda:0


2023-09-09 14:42:22,448 - modelscope - WARNING - No preprocessor field found in cfg.
2023-09-09 14:42:22,450 - modelscope - WARNING - No val key and type key found in preprocessor domain of configuration.json file.
2023-09-09 14:42:22,452 - modelscope - WARNING - Cannot find available config to build preprocessor at mode inference, current config: {'model_dir': '/home/yangwenhao/.cache/modelscope/hub/damo/speech_ecapa-tdnn_sv_en_voxceleb_16k'}. trying to build by task and model information.
2023-09-09 14:42:22,453 - modelscope - WARNING - No preprocessor key ('ecapa-tdnn-sv', 'speaker-verification') found in PREPROCESSOR_MAP, skip building preprocessor.


In [1]:
from modelscope.pipelines import pipeline
sv_pipeline = pipeline(
    task='speaker-verification',
    model='damo/speech_campplus_sv_zh-cn_16k-common',
    model_revision='v1.0.0'
)

2023-09-11 12:18:24,769 - modelscope - INFO - PyTorch version 2.0.1 Found.
2023-09-11 12:18:24,771 - modelscope - INFO - Loading ast index from /home/yangwenhao/.cache/modelscope/ast_indexer
2023-09-11 12:18:24,962 - modelscope - INFO - Loading done! Current index file version is 1.9.0, with md5 2fd7920946625a585615c128019784ec and a total number of 921 components indexed
/home/yangwenhao/anaconda3/envs/model/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-09-11 12:18:25,883 - modelscope - INFO - Use user-specified model revision: v1.0.0
Downloading: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 26.7M/26.7M [01:55<00:00, 242kB/s]
Downloading: 100%|█████

In [8]:
from tqdm import tqdm
import h5py
import torch
from Eval.eval_metrics import evaluate_kaldi_eer, evaluate_kaldi_mindcf

def read_hdf5(reader, key):
    with h5py.File(reader, 'r') as r:
        data_flat = r.get(key)[:]
        return data_flat

In [5]:
test_subset = 'test_orgchn2band' # 'test_orgchn2'
save_dir    = 'radio/data/cam_pp_zh/'
wav_scp     = '/home/yangwenhao/project/lstm_speaker_verification/data/vox1/%s/wav.scp'%(test_subset)

uid2path = {}
emb_file = save_dir + 'xvector.h5py'

with  h5py.File(emb_file, 'w') as gf:
    with open(wav_scp, 'r') as f:
        for l in tqdm(f.readlines(), ncols=50):
            uid, upath = l.split()
            uid2path[uid] = upath

            result = sv_pipeline(in_audios=[upath], output_emb=True)
            emb = result[1]

            gf.create_dataset(uid, data=emb, compression="gzip")

100%|███████| 14622/14622 [12:50<00:00, 18.97it/s]


In [7]:
dist_fn = torch.nn.CosineSimilarity()

In [8]:
# for subs in ['bandpass', 'bandcross', 'bandradio']:
for subs in ['original', 'radio', 'radiocross', 'bandpass', 'bandcross', 'bandradio']:
    
    trials = '/home/yangwenhao/project/lstm_speaker_verification/data/vox1/%s/trials_%s'%(test_subset, subs)
    distances = []
    labels = []
    with open(trials, 'r') as f:
        for l in tqdm(f.readlines(), ncols=50):
            a,b,target = l.split()
            a = torch.tensor(read_hdf5(emb_file, a))
            b = torch.tensor(read_hdf5(emb_file, b))
            d = dist_fn(a, b)
            distances.append(d[0])

            if target == 'target':
                labels.append(True)
            else:
                labels.append(False)

    eer, eer_threshold, accuracy = evaluate_kaldi_eer(
            distances, labels, cos=True, re_thre=True)
    mindcf_01, mindcf_001 = evaluate_kaldi_mindcf(distances, labels)

    print('{:<12s}:  EER(%): {:.2f}  MinDCF: {:.4f}'.format(subs, eer*100, mindcf_01))
    
# cam_pp vox2
# original    :  EER(%): 0.89  MinDCF: 0.0938
# radio       :  EER(%): 2.35  MinDCF: 0.3141
# radiocross  :  EER(%): 2.10  MinDCF: 0.2678

# ECAPA-TDNN vox2
# original    :  EER(%): 1.00  MinDCF: 0.1021
# radio       :  EER(%): 2.70  MinDCF: 0.3496
# radiocross  :  EER(%): 2.31  MinDCF: 0.3055
# bandpass    :  EER(%): 3.05  MinDCF: 0.3696
# bandcross   :  EER(%): 2.31  MinDCF: 0.3072
# bandradio   :  EER(%): 3.36  MinDCF: 0.4165

# cam_pp 3d-zh
# original    :  EER(%): 7.88  MinDCF: 0.6422
# radio       :  EER(%): 21.27  MinDCF: 0.9945
# radiocross  :  EER(%): 24.14  MinDCF: 0.9264
# bandpass    :  EER(%): 17.62  MinDCF: 0.8550
# bandcross   :  EER(%): 17.66  MinDCF: 0.9662
# bandradio   :  EER(%): 26.63  MinDCF: 0.9773

100%|██████| 37720/37720 [00:54<00:00, 694.65it/s]


original    :  EER(%): 7.88  MinDCF: 0.6422


100%|██████| 37720/37720 [00:52<00:00, 721.05it/s]


radio       :  EER(%): 21.27  MinDCF: 0.9945


100%|██████| 75440/75440 [01:45<00:00, 713.43it/s]


...

bandpass    :  EER(%): 17.62  MinDCF: 0.8550


100%|██████| 75440/75440 [01:45<00:00, 715.75it/s]


bandcross   :  EER(%): 17.66  MinDCF: 0.9662


100%|██████| 75440/75440 [01:45<00:00, 716.91it/s]


bandradio   :  EER(%): 26.63  MinDCF: 0.9773
